In [3]:
-#export
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

MNIST_URL='https://github.com/mnielsen/rmnist/blob/master/data/mnist.pkl.gz?raw=true'

In [4]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s



In [5]:
x_train,y_train,x_valid,y_valid = get_data()

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()

In [7]:
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [8]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(2.1624e-08), tensor(1.))

In [9]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [10]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [11]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [12]:
nh=50

In [13]:
nh = 50
w1 = torch.randn(m,nh)*math.sqrt(2/m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)


In [14]:
def lin(x, w, b): return x@w + b

In [15]:
def relu(x): return x.clamp_min(0)

In [16]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0507))

In [17]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5297), tensor(0.8213))

In [18]:
#export
from torch.nn import init

In [19]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode="fan_out")
t = relu(lin(x_valid, w1, b1))

In [20]:
w1.mean(), w1.std()

(tensor(0.0003), tensor(0.0505))

In [21]:
t.mean(), t.std()

(tensor(0.5719), tensor(0.8301))

In [22]:
w1.shape

torch.Size([784, 50])

In [24]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [25]:
%timeit -n 10 _=model(x_valid)

10.4 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
model(x_valid).T.shape

torch.Size([1, 10000])

In [27]:
def mse(output, targ): return( output.squeeze(-1)-targ ).pow(2).mean()

In [28]:
y_train, y_valid = y_train.float(), y_valid.float()

In [29]:
preds = model(x_train)

In [30]:
preds.shape

torch.Size([50000, 1])

In [31]:
mse(preds, y_train)

tensor(17.2909)

In [32]:


class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)



class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g



class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)



class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]


class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()



In [36]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()



In [37]:

%time loss = model(x_train, y_train)

CPU times: user 195 ms, sys: 0 ns, total: 195 ms
Wall time: 52.9 ms


In [38]:
%time model.backward()


CPU times: user 381 ms, sys: 89.2 ms, total: 470 ms
Wall time: 121 ms


In [39]:
from torch import nn


class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse 
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

model = Model(m, nh, 1)

In [43]:
%time loss = model(x_train, y_train)

CPU times: user 206 ms, sys: 0 ns, total: 206 ms
Wall time: 78.2 ms


In [44]:
%time loss.backward()

CPU times: user 217 ms, sys: 0 ns, total: 217 ms
Wall time: 54.7 ms


In [32]:
import torch.nn as nn

In [128]:
def mse(y, t): return (y.squeeze()-t).pow(2).mean()

class Relu(nn.Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g


class Model(nn.Module):
    def __init__(self):
        self.layers = [
            nn.Linear(250, 120),
            Relu(),
            nn.Linear(120, 84),
            Relu(),
            nn.Linear(84, 10)
        ]
        self.loss = mse

    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()
    def update(self, lr):
        for l in self.layers:
            if hasattr(l, 'w'): l.w.data -= lr * l.w.grad
            if hasattr(l, 'b'): l.b.data -= lr * l.b.grad


In [129]:
model = Model()

In [130]:
X = torch.randn(250)
print(X.mean())
print(X.std())
for layer in model.layers:
    X = layer(X)
    
    print(layer.__class__.__name__, '   ' + '\t',X.shape)
    print('Mean          ', X.mean())
    print('Std           ', X.std())

tensor(0.0529)
tensor(1.1276)
Linear    	 torch.Size([120])
Mean           tensor(0.0037, grad_fn=<MeanBackward0>)
Std            tensor(0.6605, grad_fn=<StdBackward0>)
Relu    	 torch.Size([120])
Mean           tensor(-0.2277, grad_fn=<MeanBackward0>)
Std            tensor(0.3480, grad_fn=<StdBackward0>)
Linear    	 torch.Size([84])
Mean           tensor(-0.0542, grad_fn=<MeanBackward0>)
Std            tensor(0.2351, grad_fn=<StdBackward0>)
Relu    	 torch.Size([84])
Mean           tensor(-0.4325, grad_fn=<MeanBackward0>)
Std            tensor(0.1263, grad_fn=<StdBackward0>)
Linear    	 torch.Size([10])
Mean           tensor(0.0845, grad_fn=<MeanBackward0>)
Std            tensor(0.3006, grad_fn=<StdBackward0>)


In [136]:
torch.nn.Linear??

Init signature:
torch.nn.Linear(
    in_features: int,
    out_features: int,
    bias: bool = True,
    device=None,
    dtype=None,
) -> None
Source:        
class Linear(Module):
    r"""Applies a linear transformation to the incoming data: :math:`y = xA^T + b`

    This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to ``False``, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(*, H_{in})` where :math:`*` means any number of
          dimensions including none and :math:`H_{in} = \text{in\_features}`.
        - Output: :math:`(*, H_{out})` where all but the last dimension
          are the same shape as the input and :math:`H_{out} = \text{out\_features}`.

    Attributes:
        weight: the learnable weights of the module of shape
            :math:`(\text{out\_features}, \text{in\_featu